# 0. Install and Import Dependencies

In [1]:
!pip install mediapipe opencv-python pandas scikit-learn

In [2]:
import mediapipe as mp # Import mediapipe
import cv2 # Import opencv
import numpy as np
import csv
import os
import pandas as pd
import pickle


In [3]:
mp_drawing = mp.solutions.drawing_utils # Drawing helpers
mp_holistic = mp.solutions.holistic # Mediapipe Solutions
mp_pose = mp.solutions.pose

# 4. Make Detections with Model

In [4]:
with open('gym_pose.pkl', 'rb') as f:
    model = pickle.load(f)

/Users/hyungjucha/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator StandardScaler from version 0.24.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/Users/hyungjucha/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/Users/hyungjucha/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/Users/hyungjucha/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator Pipeline from version 0.24.2 w

In [5]:
model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier', RandomForestClassifier())])

In [6]:
#팔, 어깨 각도 계산
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [7]:
cap = cv2.VideoCapture(0)

# ankle position detector variables
ankle_pos = None
counter = 0
stage= None

# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 2. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 3. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        # Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Concatenate rows
            row = pose_row
            
            
#             # Append class name 
#             row.insert(0, class_name)
            
#             # Export to CSV
#             with open('coords.csv', mode='a', newline='') as f:
#                 csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
#                 csv_writer.writerow(row) 
            
            
            # Make Detections
            X = pd.DataFrame([row])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            print(body_language_class, body_language_prob)
            
            # Grab ear coords
            coords = tuple(np.multiply(
                            np.array(
                                (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                 results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                        , [640,480]).astype(int))
            
            cv2.rectangle(image, 
                          (coords[0], coords[1]+5), 
                          (coords[0]+len(body_language_class)*20, coords[1]-30), 
                          (245, 117, 16), -1)
            cv2.putText(image, body_language_class, coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            #Get status box
            cv2.rectangle(image, (0,0), (700, 60), (245, 117, 16), -1)
            
            #Display Class
            cv2.putText(image, 'CLASS'
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, body_language_class.split(' ')[0]
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            #Display Probability
            cv2.putText(image, 'PROB'
                       , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                       , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Get coordinates
            shoulder = [pose[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,pose[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [pose[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,pose[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [pose[mp_pose.PoseLandmark.LEFT_WRIST.value].x,pose[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            thumb = [pose[mp_pose.PoseLandmark.LEFT_THUMB.value].x,pose[mp_pose.PoseLandmark.LEFT_THUMB.value].y]
            hip = [pose[mp_pose.PoseLandmark.LEFT_HIP.value].x,pose[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            
            # Calculate angle
            angle = calculate_angle(shoulder, elbow, wrist)
            angle2 = calculate_angle(elbow, wrist, thumb)
            angle3 = calculate_angle(shoulder, elbow, hip)
            
            # Visualize angle
            cv2.putText(image, str(angle), 
                           tuple(np.multiply(elbow, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
            
            # Rep counting logic
          
            if angle > 160:
                stage = "down"
            if angle < 50 and stage =='down':
                stage="up"
                counter +=1
                print(counter)
            
            
                
            
            # ankle counter logic
            if angle2 < 130:
                ankle_pos = "ankle twisted"
            else:
                ankle_pos= None
                
              # Ankle data
            cv2.putText(image, ankle_pos, 
                        (10,200), 
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (0,0,255), 2, cv2.LINE_AA)
                
            # Rep data
            cv2.putText(image, 'REPS', (400,12), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, str(counter), 
                        (400,60), 
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)

            # Stage data
            cv2.putText(image, 'STAGE', (500,12), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, stage, 
                        (500,60), 
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
            
            # Render detections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                    mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                     )                     
               
        
        except:
            pass  
            
        
        cv2.imshow('Gym-pose decoder', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

nice pushup [0.31 0.24 0.04 0.38 0.03]
nice pushup [0.29 0.29 0.04 0.35 0.03]
nice deadlift [0.25 0.34 0.04 0.34 0.03]
nice deadlift [0.25 0.34 0.04 0.34 0.03]
nice deadlift [0.24 0.38 0.02 0.33 0.03]
nice deadlift [0.24 0.38 0.02 0.33 0.03]
nice deadlift [0.24 0.35 0.04 0.34 0.03]
nice pushup [0.27 0.29 0.05 0.36 0.03]
nice pushup [0.27 0.29 0.05 0.36 0.03]
nice pushup [0.27 0.29 0.04 0.37 0.03]
nice pushup [0.27 0.28 0.06 0.36 0.03]
nice pushup [0.28 0.26 0.07 0.36 0.03]
nice pushup [0.24 0.25 0.07 0.41 0.03]
nice pushup [0.24 0.24 0.06 0.43 0.03]
nice pushup [0.24 0.24 0.06 0.43 0.03]
nice pushup [0.25 0.23 0.06 0.43 0.03]
nice pushup [0.28 0.19 0.06 0.44 0.03]
nice pushup [0.26 0.21 0.06 0.44 0.03]
nice pushup [0.23 0.25 0.05 0.44 0.03]
nice pushup [0.2  0.29 0.06 0.42 0.03]
nice deadlift [0.14 0.47 0.   0.37 0.02]
nice deadlift [0.19 0.4  0.   0.37 0.04]
nice deadlift [0.18 0.38 0.   0.35 0.09]
nice deadlift [0.19 0.34 0.   0.28 0.19]
nice deadlift [0.2  0.34 0.   0.26 0.2 ]
nice 

nice benchpress [0.9  0.03 0.02 0.05 0.  ]
nice benchpress [0.9  0.03 0.02 0.05 0.  ]
nice benchpress [0.89 0.03 0.02 0.05 0.01]
nice benchpress [0.77 0.06 0.   0.13 0.04]
1
nice benchpress [0.79 0.05 0.   0.11 0.05]
nice benchpress [0.8  0.05 0.   0.11 0.04]
nice benchpress [0.76 0.04 0.04 0.12 0.04]
nice benchpress [0.76 0.04 0.05 0.12 0.03]
nice benchpress [0.77 0.02 0.05 0.13 0.03]
nice benchpress [0.76 0.03 0.05 0.13 0.03]
nice benchpress [0.76 0.03 0.05 0.13 0.03]
nice benchpress [0.74 0.04 0.04 0.14 0.04]
nice benchpress [0.7  0.05 0.03 0.17 0.05]
nice benchpress [0.68 0.06 0.04 0.18 0.04]
nice benchpress [0.62 0.05 0.02 0.27 0.04]
nice benchpress [0.61 0.04 0.02 0.27 0.06]
nice benchpress [0.57 0.04 0.02 0.23 0.14]
nice benchpress [0.55 0.03 0.02 0.27 0.13]
nice benchpress [0.56 0.04 0.03 0.23 0.14]
nice benchpress [0.56 0.05 0.03 0.19 0.17]
nice benchpress [0.59 0.03 0.03 0.13 0.22]
nice benchpress [0.63 0.03 0.01 0.17 0.16]
nice benchpress [0.64 0.03 0.   0.17 0.16]
nice benc

nice benchpress [0.49 0.06 0.34 0.07 0.04]
nice benchpress [0.49 0.06 0.34 0.07 0.04]
nice benchpress [0.5  0.04 0.35 0.07 0.04]
nice benchpress [0.62 0.08 0.23 0.02 0.05]
nice benchpress [0.73 0.04 0.19 0.   0.04]
nice benchpress [0.56 0.07 0.16 0.01 0.2 ]
nice benchpress [0.56 0.08 0.14 0.03 0.19]
nice benchpress [0.58 0.08 0.15 0.04 0.15]
nice benchpress [0.64 0.08 0.17 0.06 0.05]
nice benchpress [0.68 0.07 0.15 0.05 0.05]
nice benchpress [0.69 0.07 0.14 0.05 0.05]
nice benchpress [0.68 0.07 0.16 0.04 0.05]
nice benchpress [0.67 0.06 0.16 0.06 0.05]
nice benchpress [0.67 0.06 0.17 0.05 0.05]
nice benchpress [0.58 0.07 0.13 0.01 0.21]
nice benchpress [0.62 0.06 0.12 0.01 0.19]
nice benchpress [0.71 0.07 0.14 0.03 0.05]
nice benchpress [0.69 0.07 0.14 0.04 0.06]
nice benchpress [0.68 0.07 0.14 0.05 0.06]
nice benchpress [0.67 0.07 0.14 0.06 0.06]
nice benchpress [0.67 0.07 0.14 0.06 0.06]
nice benchpress [0.68 0.07 0.13 0.06 0.06]
nice benchpress [0.69 0.07 0.12 0.06 0.06]
nice benchp

nice deadlift [0.32 0.48 0.1  0.09 0.01]
nice deadlift [0.3  0.47 0.11 0.11 0.01]
nice deadlift [0.32 0.43 0.12 0.12 0.01]
nice benchpress [0.37 0.37 0.11 0.14 0.01]
nice benchpress [0.39 0.28 0.16 0.16 0.01]
nice benchpress [0.43 0.21 0.19 0.16 0.01]
nice benchpress [0.41 0.2  0.23 0.15 0.01]
nice benchpress [0.39 0.2  0.24 0.14 0.03]
nice benchpress [0.4  0.2  0.21 0.1  0.09]
3
nice benchpress [0.41 0.13 0.26 0.11 0.09]
nice benchpress [0.41 0.13 0.26 0.11 0.09]
nice benchpress [0.41 0.13 0.27 0.1  0.09]
nice benchpress [0.51 0.14 0.26 0.08 0.01]
nice benchpress [0.47 0.13 0.28 0.11 0.01]
nice benchpress [0.51 0.14 0.27 0.07 0.01]
nice benchpress [0.46 0.13 0.29 0.11 0.01]
nice benchpress [0.51 0.14 0.25 0.09 0.01]
nice benchpress [0.45 0.13 0.28 0.12 0.02]
nice benchpress [0.41 0.13 0.27 0.1  0.09]
nice benchpress [0.4  0.13 0.27 0.11 0.09]
nice benchpress [0.44 0.17 0.25 0.1  0.04]
nice benchpress [0.43 0.19 0.24 0.1  0.04]
nice benchpress [0.46 0.19 0.23 0.09 0.03]
nice benchpress

nice benchpress [0.46 0.05 0.07 0.32 0.1 ]
nice benchpress [0.45 0.07 0.07 0.28 0.13]
nice benchpress [0.49 0.06 0.05 0.29 0.11]
nice benchpress [0.52 0.06 0.05 0.26 0.11]
nice benchpress [0.53 0.07 0.04 0.25 0.11]
nice benchpress [0.58 0.08 0.02 0.21 0.11]
nice benchpress [0.51 0.12 0.02 0.2  0.15]
6
nice benchpress [0.54 0.12 0.02 0.18 0.14]
nice benchpress [0.52 0.12 0.01 0.18 0.17]
nice benchpress [0.5  0.12 0.03 0.16 0.19]
nice benchpress [0.48 0.12 0.04 0.21 0.15]
nice benchpress [0.48 0.13 0.03 0.23 0.13]
nice benchpress [0.47 0.13 0.04 0.19 0.17]
nice squat [0.15 0.27 0.04 0.05 0.49]
nice deadlift [0.21 0.44 0.   0.07 0.28]
nice squat [0.17 0.24 0.05 0.04 0.5 ]
nice squat [0.18 0.25 0.05 0.03 0.49]
nice deadlift [0.19 0.41 0.03 0.07 0.3 ]
nice deadlift [0.19 0.42 0.02 0.06 0.31]
nice deadlift [0.19 0.48 0.   0.1  0.23]
nice deadlift [0.17 0.51 0.   0.04 0.28]
nice deadlift [0.19 0.5  0.   0.03 0.28]
nice deadlift [0.18 0.49 0.01 0.03 0.29]
nice deadlift [0.16 0.5  0.   0.16 0.1

...

nice deadlift [0.27 0.67 0.05 0.01 0.  ]
nice deadlift [0.27 0.64 0.07 0.01 0.01]
nice deadlift [0.28 0.63 0.07 0.01 0.01]
nice deadlift [0.26 0.67 0.05 0.01 0.01]
nice deadlift [0.29 0.65 0.04 0.01 0.01]
nice deadlift [0.24 0.66 0.07 0.01 0.02]
nice deadlift [0.23 0.65 0.08 0.   0.04]
nice deadlift [0.22 0.61 0.09 0.   0.08]
nice deadlift [0.23 0.57 0.09 0.   0.11]
nice deadlift [0.21 0.48 0.07 0.01 0.23]
nice deadlift [0.25 0.61 0.07 0.01 0.06]
nice deadlift [0.26 0.6  0.08 0.01 0.05]
nice deadlift [0.26 0.6  0.08 0.01 0.05]
nice deadlift [0.28 0.6  0.08 0.01 0.03]
nice deadlift [0.27 0.6  0.09 0.01 0.03]
nice deadlift [0.25 0.61 0.09 0.02 0.03]
nice deadlift [0.25 0.6  0.1  0.02 0.03]
nice deadlift [0.25 0.6  0.1  0.02 0.03]
nice deadlift [0.25 0.59 0.11 0.02 0.03]
nice deadlift [0.24 0.61 0.12 0.01 0.02]
nice deadlift [0.25 0.59 0.12 0.01 0.03]
nice deadlift [0.26 0.6  0.11 0.   0.03]
nice deadlift [0.26 0.59 0.12 0.   0.03]
nice deadlift [0.24 0.51 0.15 0.01 0.09]
nice deadlift [0

nice benchpress [0.4  0.08 0.02 0.13 0.37]
13
nice benchpress [0.51 0.06 0.02 0.16 0.25]
nice benchpress [0.65 0.07 0.05 0.1  0.13]
nice benchpress [0.46 0.04 0.01 0.18 0.31]
nice benchpress [0.7  0.02 0.   0.1  0.18]
nice benchpress [0.69 0.03 0.   0.07 0.21]
nice benchpress [0.72 0.02 0.   0.07 0.19]
nice benchpress [0.69 0.04 0.02 0.02 0.23]
nice benchpress [0.69 0.03 0.   0.06 0.22]
nice benchpress [0.72 0.02 0.   0.06 0.2 ]
nice benchpress [0.71 0.03 0.   0.05 0.21]
nice benchpress [0.71 0.03 0.   0.05 0.21]
nice benchpress [0.72 0.03 0.   0.04 0.21]
nice benchpress [0.73 0.02 0.   0.06 0.19]
14
nice benchpress [0.73 0.02 0.   0.03 0.22]
nice benchpress [0.62 0.02 0.   0.16 0.2 ]
nice benchpress [0.62 0.03 0.   0.16 0.19]
nice benchpress [0.62 0.02 0.   0.17 0.19]
nice squat [0.39 0.01 0.   0.1  0.5 ]
nice benchpress [0.61 0.03 0.   0.17 0.19]
nice benchpress [0.59 0.03 0.   0.19 0.19]
nice benchpress [0.55 0.01 0.   0.23 0.21]
nice benchpress [0.57 0.03 0.   0.15 0.25]
nice bench

nice squat [0.26 0.04 0.09 0.05 0.56]
nice squat [0.31 0.03 0.07 0.05 0.54]
nice squat [0.27 0.03 0.08 0.06 0.56]
nice squat [0.26 0.03 0.09 0.06 0.56]
nice squat [0.24 0.04 0.09 0.06 0.57]
nice squat [0.24 0.04 0.09 0.06 0.57]
nice squat [0.24 0.05 0.09 0.06 0.56]
nice squat [0.25 0.05 0.1  0.06 0.54]
nice squat [0.29 0.04 0.11 0.05 0.51]
nice squat [0.29 0.04 0.11 0.05 0.51]
nice squat [0.29 0.03 0.12 0.05 0.51]
nice squat [0.28 0.03 0.13 0.06 0.5 ]
nice benchpress [0.39 0.03 0.19 0.06 0.33]
nice benchpress [0.38 0.03 0.2  0.06 0.33]
nice benchpress [0.44 0.03 0.23 0.03 0.27]
nice benchpress [0.4  0.03 0.2  0.04 0.33]
nice benchpress [0.43 0.03 0.18 0.03 0.33]
nice benchpress [0.46 0.03 0.2  0.03 0.28]
nice benchpress [0.45 0.06 0.33 0.03 0.13]
nice benchpress [0.46 0.06 0.36 0.04 0.08]
nice benchpress [0.5  0.06 0.34 0.04 0.06]
nice benchpress [0.51 0.07 0.34 0.03 0.05]
nice benchpress [0.49 0.06 0.36 0.04 0.05]
nice benchpress [0.5  0.06 0.35 0.04 0.05]
nice benchpress [0.51 0.05 0

nice benchpress [0.45 0.13 0.25 0.11 0.06]
nice benchpress [0.44 0.13 0.23 0.14 0.06]
nice benchpress [0.43 0.13 0.25 0.15 0.04]
nice benchpress [0.43 0.13 0.25 0.15 0.04]
nice benchpress [0.43 0.13 0.25 0.15 0.04]
nice benchpress [0.39 0.16 0.27 0.18 0.  ]
nice benchpress [0.39 0.16 0.29 0.16 0.  ]
nice pullup [0.31 0.05 0.44 0.09 0.11]
25
nice pullup [0.33 0.11 0.35 0.12 0.09]
nice benchpress [0.36 0.13 0.32 0.11 0.08]
nice benchpress [0.35 0.13 0.33 0.11 0.08]
nice benchpress [0.35 0.12 0.34 0.11 0.08]
nice benchpress [0.36 0.13 0.31 0.12 0.08]
nice pullup [0.33 0.06 0.39 0.11 0.11]
nice pullup [0.32 0.03 0.41 0.12 0.12]
nice benchpress [0.43 0.16 0.27 0.11 0.03]
nice benchpress [0.44 0.15 0.24 0.1  0.07]
nice benchpress [0.42 0.16 0.25 0.11 0.06]
nice benchpress [0.43 0.16 0.25 0.12 0.04]
nice benchpress [0.43 0.16 0.26 0.11 0.04]
nice benchpress [0.43 0.16 0.26 0.11 0.04]
nice benchpress [0.41 0.16 0.27 0.11 0.05]
nice benchpress [0.38 0.12 0.19 0.13 0.18]
nice benchpress [0.39 0.

nice benchpress [0.51 0.17 0.08 0.15 0.09]
nice benchpress [0.55 0.18 0.09 0.11 0.07]
nice benchpress [0.54 0.2  0.08 0.11 0.07]
nice benchpress [0.47 0.19 0.07 0.16 0.11]
nice benchpress [0.48 0.19 0.06 0.17 0.1 ]
nice benchpress [0.41 0.21 0.05 0.2  0.13]
nice benchpress [0.39 0.21 0.06 0.21 0.13]
nice benchpress [0.42 0.21 0.05 0.2  0.12]
nice benchpress [0.42 0.21 0.05 0.2  0.12]
nice benchpress [0.42 0.21 0.05 0.2  0.12]
nice benchpress [0.42 0.21 0.05 0.2  0.12]
nice benchpress [0.42 0.21 0.05 0.2  0.12]
nice benchpress [0.42 0.21 0.05 0.2  0.12]
nice benchpress [0.42 0.21 0.05 0.2  0.12]
nice benchpress [0.42 0.21 0.05 0.2  0.12]
nice benchpress [0.42 0.21 0.05 0.2  0.12]
nice benchpress [0.42 0.21 0.05 0.2  0.12]
nice benchpress [0.42 0.21 0.05 0.2  0.12]
nice benchpress [0.42 0.21 0.05 0.2  0.12]
nice benchpress [0.45 0.18 0.05 0.2  0.12]
nice benchpress [0.49 0.16 0.05 0.2  0.1 ]
nice benchpress [0.48 0.14 0.06 0.2  0.12]
nice benchpress [0.49 0.14 0.05 0.2  0.12]
nice benchp

In [8]:
import numpy as np

tuple(np.multiply(np.array((results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y)), [640,480]).astype(int))

(410, 5)

In [9]:
###Deadlift rep counter

In [11]:
cap = cv2.VideoCapture(0)

# ankle position detector variables
ankle_pos = None
counter = 0
stage= None

# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 2. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 3. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        # Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Concatenate rows
            row = pose_row
            
            
#             # Append class name 
#             row.insert(0, class_name)
            
#             # Export to CSV
#             with open('coords.csv', mode='a', newline='') as f:
#                 csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
#                 csv_writer.writerow(row) 
            
            
            # Make Detections
            X = pd.DataFrame([row])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            print(body_language_class, body_language_prob)
            
            # Grab ear coords
            coords = tuple(np.multiply(
                            np.array(
                                (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                 results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                        , [640,480]).astype(int))
            
            cv2.rectangle(image, 
                          (coords[0], coords[1]+5), 
                          (coords[0]+len(body_language_class)*20, coords[1]-30), 
                          (245, 117, 16), -1)
            cv2.putText(image, body_language_class, coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            #Get status box
            cv2.rectangle(image, (0,0), (700, 60), (245, 117, 16), -1)
            
            #Display Class
            cv2.putText(image, 'CLASS'
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, body_language_class.split(' ')[0]
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            #Display Probability
            cv2.putText(image, 'PROB'
                       , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                       , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Get coordinates
            shoulder = [pose[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,pose[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [pose[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,pose[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [pose[mp_pose.PoseLandmark.LEFT_WRIST.value].x,pose[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            thumb = [pose[mp_pose.PoseLandmark.LEFT_THUMB.value].x,pose[mp_pose.PoseLandmark.LEFT_THUMB.value].y]
            hip = [pose[mp_pose.PoseLandmark.LEFT_HIP.value].x,pose[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            
            # Calculate angle
            angle = calculate_angle(shoulder, elbow, hip)
            
            # Visualize angle
            cv2.putText(image, str(angle), 
                           tuple(np.multiply(elbow, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            
            
            # Rep counting logic
          
            if angle > 150:
                stage = "up"
            if angle < 80 and stage =='up':
                stage="down"
                counter +=1
                print(counter)
            
            
            # ankle counter logic
            if angle2 < 130:
                ankle_pos = "ankle twisted"
            else:
                ankle_pos= None
                
              # Ankle data
            cv2.putText(image, ankle_pos, 
                        (10,200), 
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (0,0,255), 2, cv2.LINE_AA)
                
            # Rep data
            cv2.putText(image, 'REPS', (400,12), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, str(counter), 
                        (400,60), 
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)

            # Stage data
            cv2.putText(image, 'STAGE', (500,12), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, stage, 
                        (500,60), 
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
            
            # Render detections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                    mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                     )                     
               
        
        except:
            pass  
            
        
        cv2.imshow('Gym-pose decoder', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

nice benchpress [0.45 0.27 0.06 0.17 0.05]
nice benchpress [0.46 0.27 0.06 0.14 0.07]
nice benchpress [0.45 0.27 0.05 0.15 0.08]
nice benchpress [0.41 0.23 0.03 0.22 0.11]
nice benchpress [0.42 0.23 0.03 0.22 0.1 ]
nice benchpress [0.42 0.23 0.03 0.22 0.1 ]
nice benchpress [0.42 0.24 0.03 0.22 0.09]
nice benchpress [0.42 0.23 0.03 0.22 0.1 ]
nice benchpress [0.44 0.23 0.03 0.2  0.1 ]
nice benchpress [0.44 0.24 0.03 0.19 0.1 ]
nice benchpress [0.42 0.25 0.04 0.19 0.1 ]
nice benchpress [0.41 0.26 0.04 0.19 0.1 ]
nice benchpress [0.41 0.26 0.04 0.19 0.1 ]
nice benchpress [0.41 0.26 0.04 0.19 0.1 ]
nice benchpress [0.41 0.26 0.04 0.19 0.1 ]
nice benchpress [0.41 0.26 0.04 0.19 0.1 ]
nice benchpress [0.41 0.26 0.04 0.19 0.1 ]
nice benchpress [0.41 0.27 0.04 0.19 0.09]
nice benchpress [0.41 0.27 0.04 0.19 0.09]
nice deadlift [0.34 0.41 0.02 0.16 0.07]
nice deadlift [0.26 0.51 0.01 0.15 0.07]
nice deadlift [0.25 0.52 0.01 0.13 0.09]
nice deadlift [0.27 0.53 0.   0.11 0.09]
nice deadlift [0.27

nice benchpress [0.61 0.01 0.11 0.04 0.23]
nice benchpress [0.64 0.01 0.08 0.04 0.23]
nice benchpress [0.6  0.01 0.11 0.05 0.23]
nice benchpress [0.58 0.01 0.1  0.06 0.25]
nice benchpress [0.66 0.02 0.12 0.04 0.16]
nice benchpress [0.6  0.03 0.2  0.1  0.07]
nice benchpress [0.63 0.03 0.2  0.08 0.06]
nice benchpress [0.63 0.03 0.2  0.07 0.07]
nice benchpress [0.56 0.03 0.22 0.08 0.11]
nice benchpress [0.69 0.04 0.15 0.07 0.05]
nice benchpress [0.56 0.03 0.18 0.07 0.16]
nice benchpress [0.53 0.05 0.17 0.11 0.14]
nice benchpress [0.47 0.04 0.08 0.25 0.16]
4
nice squat [0.38 0.04 0.06 0.12 0.4 ]
nice squat [0.32 0.05 0.04 0.16 0.43]
nice squat [0.3  0.01 0.07 0.07 0.55]
nice squat [0.3  0.01 0.11 0.05 0.53]
nice squat [0.16 0.04 0.04 0.04 0.72]
nice squat [0.24 0.04 0.07 0.13 0.52]
nice squat [0.31 0.05 0.06 0.15 0.43]
nice squat [0.28 0.04 0.03 0.07 0.58]
nice squat [0.23 0.01 0.06 0.11 0.59]
nice squat [0.27 0.06 0.07 0.07 0.53]
5
nice squat [0.05 0.01 0.01 0.04 0.89]
nice squat [0.2  0.

nice benchpress [0.66 0.04 0.23 0.04 0.03]
nice benchpress [0.64 0.07 0.22 0.04 0.03]
nice benchpress [0.66 0.04 0.25 0.03 0.02]
nice benchpress [0.68 0.02 0.25 0.03 0.02]
nice benchpress [0.69 0.02 0.24 0.03 0.02]
nice benchpress [0.68 0.04 0.22 0.03 0.03]
nice benchpress [0.69 0.04 0.21 0.03 0.03]
nice benchpress [0.69 0.02 0.23 0.03 0.03]
nice benchpress [0.7  0.03 0.2  0.03 0.04]
nice benchpress [0.7  0.06 0.2  0.01 0.03]
nice benchpress [0.7  0.06 0.2  0.01 0.03]
nice benchpress [0.68 0.06 0.21 0.02 0.03]
nice benchpress [0.68 0.06 0.21 0.02 0.03]
nice benchpress [0.68 0.07 0.2  0.02 0.03]
nice benchpress [0.65 0.07 0.2  0.04 0.04]
nice benchpress [0.66 0.05 0.2  0.05 0.04]
nice benchpress [0.66 0.05 0.21 0.04 0.04]
nice benchpress [0.66 0.05 0.22 0.04 0.03]
nice benchpress [0.69 0.05 0.22 0.02 0.02]
nice benchpress [0.66 0.05 0.22 0.04 0.03]
nice benchpress [0.66 0.05 0.22 0.04 0.03]
nice benchpress [0.62 0.07 0.23 0.05 0.03]
nice benchpress [0.62 0.07 0.23 0.05 0.03]
nice benchp

nice squat [0.04 0.34 0.04 0.   0.58]
nice squat [0.04 0.41 0.06 0.   0.49]
nice deadlift [0.04 0.62 0.05 0.   0.29]
nice deadlift [0.06 0.56 0.07 0.   0.31]
nice deadlift [0.05 0.63 0.03 0.01 0.28]
nice deadlift [0.05 0.63 0.03 0.   0.29]
nice deadlift [0.07 0.6  0.04 0.01 0.28]
nice deadlift [0.06 0.57 0.07 0.   0.3 ]
nice deadlift [0.05 0.58 0.07 0.01 0.29]
nice deadlift [0.04 0.61 0.06 0.01 0.28]
nice deadlift [0.04 0.62 0.05 0.01 0.28]
nice deadlift [0.04 0.63 0.03 0.01 0.29]
nice deadlift [0.04 0.62 0.05 0.01 0.28]
nice deadlift [0.06 0.56 0.08 0.   0.3 ]
nice deadlift [0.07 0.56 0.07 0.   0.3 ]
nice deadlift [0.07 0.55 0.09 0.   0.29]
nice deadlift [0.06 0.5  0.1  0.   0.34]
nice deadlift [0.07 0.57 0.08 0.   0.28]
nice deadlift [0.08 0.56 0.07 0.   0.29]
nice deadlift [0.08 0.51 0.07 0.   0.34]
nice deadlift [0.07 0.53 0.05 0.02 0.33]
nice deadlift [0.07 0.54 0.05 0.01 0.33]
nice deadlift [0.06 0.51 0.09 0.01 0.33]
nice deadlift [0.05 0.58 0.1  0.   0.27]
nice deadlift [0.05 0.

nice squat [0.15 0.16 0.33 0.   0.36]
nice squat [0.11 0.15 0.27 0.   0.47]
nice squat [0.09 0.14 0.23 0.   0.54]
nice squat [0.11 0.14 0.21 0.   0.54]
nice squat [0.11 0.13 0.19 0.02 0.55]
nice squat [0.22 0.16 0.25 0.01 0.36]
nice squat [0.22 0.11 0.29 0.02 0.36]
nice squat [0.21 0.12 0.28 0.03 0.36]
nice pullup [0.19 0.09 0.36 0.06 0.3 ]
nice squat [0.21 0.13 0.3  0.04 0.32]
nice squat [0.26 0.15 0.21 0.07 0.31]
nice squat [0.19 0.13 0.31 0.05 0.32]
nice squat [0.22 0.16 0.2  0.12 0.3 ]
nice squat [0.15 0.29 0.17 0.09 0.3 ]
nice deadlift [0.07 0.38 0.19 0.   0.36]
nice deadlift [0.04 0.45 0.08 0.   0.43]
nice squat [0.05 0.28 0.08 0.01 0.58]
15
nice squat [0.07 0.26 0.08 0.02 0.57]
nice squat [0.07 0.26 0.08 0.02 0.57]
nice squat [0.07 0.26 0.07 0.02 0.58]
nice squat [0.08 0.27 0.07 0.02 0.56]
nice squat [0.08 0.3  0.08 0.01 0.53]
nice squat [0.08 0.33 0.1  0.01 0.48]
nice squat [0.05 0.31 0.08 0.   0.56]
nice squat [0.05 0.25 0.07 0.02 0.61]
nice squat [0.05 0.25 0.07 0.02 0.61]
ni

In [45]:
#yoga hometraining app 

In [46]:
with open('yoga_pose.pkl', 'rb') as f:
    model = pickle.load(f)

In [47]:
model

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('randomforestclassifier', RandomForestClassifier())])

In [48]:
cap = cv2.VideoCapture('b')

# ankle position detector variables
ankle_pos = None
counter = 0
stage= None

# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 1. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 2. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 3. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        # Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Concatenate rows
            row = pose_row
            
            
#             # Append class name 
#             row.insert(0, class_name)
            
#             # Export to CSV
#             with open('coords.csv', mode='a', newline='') as f:
#                 csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
#                 csv_writer.writerow(row) 
            
            
            # Make Detections
            X = pd.DataFrame([row])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            print(body_language_class, body_language_prob)
            
            # Retrieve ear coordinates 
            coords = tuple(np.multiply(
                            np.array(
                                (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                 results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                        , [640,480]).astype(int))
            
            cv2.rectangle(image, 
                          (coords[0], coords[1]+5), 
                          (coords[0]+len(body_language_class)*20, coords[1]-30), 
                          (245, 117, 16), -1)
            cv2.putText(image, body_language_class, coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            #Get status box
            cv2.rectangle(image, (0,0), (700, 60), (245, 117, 16), -1)
            
            #Display Class
            cv2.putText(image, 'CLASS'
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, body_language_class.split(' ')[0]
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            #Display Probability
            cv2.putText(image, 'PROB'
                       , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                       , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Get coordinates
            shoulder = [pose[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,pose[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [pose[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,pose[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [pose[mp_pose.PoseLandmark.LEFT_WRIST.value].x,pose[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            thumb = [pose[mp_pose.PoseLandmark.LEFT_THUMB.value].x,pose[mp_pose.PoseLandmark.LEFT_THUMB.value].y]
            
            # Calculate angle
            angle = calculate_angle(shoulder, elbow, wrist)
            angle2 = calculate_angle(elbow, wrist, thumb)
            
            # Visualize angle
            cv2.putText(image, str(angle), 
                           tuple(np.multiply(elbow, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            # Ankle data
            cv2.putText(image, ankle_pos, 
                        (10,200), 
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (0,0,255), 2, cv2.LINE_AA)
            
            # Rep counting logic
            if angle > 160:
                stage = "down"
            if angle < 30 and stage =='down':
                stage="up"
                counter +=1
                print(counter)
            
            
            # Curl counter logic
            if angle2 < 130:
                ankle_pos = "ankle twisted"
            else:
                ankle_pos= None
                
            # Rep data
            cv2.putText(image, 'REPS', (400,12), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, str(counter), 
                        (400,60), 
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)

            # Stage data
            cv2.putText(image, 'STAGE', (500,12), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, stage, 
                        (500,60), 
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
            
            # Render detections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                    mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                     )                     
               
        
        except:
            pass  
            
        
        cv2.imshow('Gym-pose decoder', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

Bhujangasana [0.92 0.   0.07 0.01 0.   0.  ]
Bhujangasana [0.9  0.   0.09 0.01 0.   0.  ]
Bhujangasana [0.9  0.   0.09 0.01 0.   0.  ]
Bhujangasana [0.9  0.   0.09 0.01 0.   0.  ]
Bhujangasana [0.9  0.   0.09 0.01 0.   0.  ]
Bhujangasana [0.9  0.   0.09 0.01 0.   0.  ]
Bhujangasana [0.91 0.   0.08 0.01 0.   0.  ]
Bhujangasana [0.91 0.   0.08 0.01 0.   0.  ]
Bhujangasana [0.91 0.   0.08 0.01 0.   0.  ]
Bhujangasana [0.91 0.   0.08 0.01 0.   0.  ]
Bhujangasana [0.91 0.   0.08 0.01 0.   0.  ]
Bhujangasana [0.91 0.   0.08 0.01 0.   0.  ]
Bhujangasana [0.91 0.   0.08 0.01 0.   0.  ]
Bhujangasana [0.91 0.   0.08 0.01 0.   0.  ]
Bhujangasana [0.92 0.   0.08 0.   0.   0.  ]
Bhujangasana [0.92 0.   0.08 0.   0.   0.  ]
Bhujangasana [0.92 0.   0.08 0.   0.   0.  ]
Bhujangasana [0.92 0.   0.08 0.   0.   0.  ]
Bhujangasana [0.92 0.   0.08 0.   0.   0.  ]
Bhujangasana [0.92 0.   0.08 0.   0.   0.  ]
Bhujangasana [0.92 0.   0.08 0.   0.   0.  ]
Bhujangasana [0.92 0.   0.08 0.   0.   0.  ]
Bhujangasa

Bhujangasana [0.96 0.   0.01 0.   0.03 0.  ]
Bhujangasana [0.96 0.   0.01 0.   0.03 0.  ]
Bhujangasana [0.96 0.   0.01 0.   0.03 0.  ]
Bhujangasana [0.96 0.   0.01 0.   0.03 0.  ]
Bhujangasana [0.96 0.   0.01 0.   0.03 0.  ]
Bhujangasana [0.94 0.   0.01 0.   0.05 0.  ]
Bhujangasana [0.93 0.   0.01 0.   0.06 0.  ]
Bhujangasana [0.93 0.   0.01 0.   0.06 0.  ]
Bhujangasana [0.93 0.   0.01 0.   0.06 0.  ]
Bhujangasana [0.93 0.   0.01 0.   0.06 0.  ]
Bhujangasana [0.93 0.   0.01 0.   0.06 0.  ]
Bhujangasana [0.93 0.   0.01 0.   0.06 0.  ]
Bhujangasana [0.93 0.   0.01 0.   0.06 0.  ]
Bhujangasana [0.93 0.   0.01 0.   0.06 0.  ]
Bhujangasana [0.93 0.   0.01 0.   0.06 0.  ]
Bhujangasana [0.93 0.   0.01 0.   0.06 0.  ]
Bhujangasana [0.93 0.   0.01 0.   0.06 0.  ]
Bhujangasana [0.93 0.   0.01 0.   0.06 0.  ]
Bhujangasana [0.93 0.   0.01 0.   0.06 0.  ]
Bhujangasana [0.93 0.   0.01 0.   0.06 0.  ]
Bhujangasana [0.93 0.   0.01 0.   0.06 0.  ]
Bhujangasana [0.93 0.   0.01 0.   0.06 0.  ]
Bhujangasa

Bhujangasana [0.95 0.   0.02 0.   0.03 0.  ]
Bhujangasana [0.95 0.   0.02 0.   0.03 0.  ]
